In [30]:
import pandas as pd
import os

# 设置数据目录和过滤阈值
data_dir = 'right_data'
min_data_points = 5  # 最小数据点数量

# 获取所有 CSV 文件
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

# 过滤并加载数据
filtered_data = []

for file in csv_files:
    file_path = os.path.join(data_dir, file)
    data = pd.read_csv(file_path)
    
    # 检查数据点数量
    if len(data) >= min_data_points:
        filtered_data.append(data)
    else:
        print(f"File {file} has only {len(data)} data points and will be ignored.")

# 检查过滤后的数据
print(f"Number of valid files: {len(filtered_data)}")


File touch_trajectory_1732895380277.csv has only 3 data points and will be ignored.
File touch_trajectory_1732895386207.csv has only 4 data points and will be ignored.
File touch_trajectory_1732895357723.csv has only 4 data points and will be ignored.
Number of valid files: 19


In [31]:
import numpy as np

processed_data = []

for data in filtered_data:
    # 计算长度
    distances = np.sqrt(np.diff(data['x'])**2 + np.diff(data['y'])**2)
    length = np.sum(distances)
    

    # 计算速度
    begin = data['timestamp'].iloc[0]
    end = data['timestamp'].iloc[-1]
    duration = (end - begin)/1000
    speed = length / duration

    # 计算位移
    displacement = np.sqrt((data['x'].iloc[-1] - data['x'].iloc[0])**2 + (data['y'].iloc[-1] - data['y'].iloc[0])**2)

    # 计算弧度
    angle = np.arctan2(data['y'].iloc[-1] - data['y'].iloc[0], data['x'].iloc[-1] - data['x'].iloc[0])

    processed_data.append([length, speed, displacement, angle])


features = np.array(processed_data)


In [32]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.nn as nn
import torch.optim as optim

# 将数据转换为 PyTorch 张量
features = torch.tensor(features, dtype=torch.float32)
dataset = TensorDataset(features)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
# 定义模型
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(4, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  #左手、右手
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# 初始化模型、损失函数和优化器
model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs = data[0]
        optimizer.zero_grad()
        outputs = model(inputs)
        # 创建标签，右手为1
        labels = torch.ones(inputs.size(0), 2, dtype=torch.float32)  # 调整标签形状以匹配输出

        # 计算损失
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, loss: {running_loss}")

# 测试模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs = data[0]
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)

        total += inputs.size(0)
        correct += (predicted == 1).sum().item()
    
print(f"Accuracy: {100 * correct / total}%")

Epoch 1, loss: 3.3941357002986905
Epoch 2, loss: 0.0002882396802306175
Epoch 3, loss: 4.023473593406379e-05
Epoch 4, loss: 3.5346787626622245e-05
Epoch 5, loss: 3.439304782659747e-05
Epoch 6, loss: 3.337969974381849e-05
Epoch 7, loss: 3.3081654692068696e-05
Epoch 8, loss: 3.248557186452672e-05
Epoch 9, loss: 3.206831024726853e-05
Epoch 10, loss: 3.177026883349754e-05
tensor([0])
tensor([0])
tensor([0])
tensor([0])
Accuracy: 0.0%
